In [2]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
library('lmmlasso')

In [3]:
source('minMod_modif.r')

In [4]:
n_subjects= 25
n_observations = 4
p = 100
q = 1

SNR_values = c(4)
prop_relevant_values = c(0.1, 0.2, 0.3)
num_iter = 50

In [5]:
prop_relevant = 0.1

In [6]:
SNR =4 
corr = 0.2

## Simulation comparing Selective Inference with minMod estimation
Comparing TPR and FDR for the two methods in different settings of SNR and proportion of non-zero coefficients. We use the original minMod and the new adapted one.

In [7]:
n_subjects= 25
n_observations = 4
p = 100
q = 1

SNR_values = c(4)
prop_relevant_values = c(0.1, 0.2, 0.3)
num_iter = 50

fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
)

rand_form = list(subjects =~ 1 + X1)

tot_its <- length(SNR_values)*length(prop_relevant_values)*num_iter

columns = c('SNR','prop_relevant', 'TPR_lasso', 'TPR_selinf', 'FDR_lasso', 'FDR_selinf', 'num_sel_lasso', 'num_sel_selinf')
results_df = data.frame(matrix(nrow = 0, ncol = length(columns))) 
colnames(results_df) = columns

pb = txtProgressBar(min = 0, max = tot_its, initial = 0, style = 3)

for(SNR in SNR_values){
    for(prop_relevant in prop_relevant_values){
        for(i in 1:num_iter){

            data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant)
            X <- data$X
            subjects <- data$subjects
            y <- data$y
            beta <- data$beta

            # fixing lambda
            lambda = max(abs(t(X) %*% y)) / 2
            

            selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

            # Now we can define the function checking the congruency
            # with the original selection
            checkFun <- function(yb){

                all(selFun(yb)$vec == sel_vec)

            }

            
            sel <- selFun(y)
            sel_vec <- sel$vec
            sel_names <- sel$names

            sel_form = as.formula(
            paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1 + X1|subjects)")
            )

            final_model = lmer(formula = sel_form, data = data.frame(X, subjects, y))
            
            # and compute valid p-values conditional on the selection
            # (this takes some time and will produce a lot of warnings)
            suppressWarnings(res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = 'minMod',
                        checkFun = checkFun, nrSamples = 100, trace = FALSE))

            sel_with_selinf <- c(rep(0,p+1))
            names(sel_with_selinf) <- names(sel_vec)
            only_sel <- names(sel_vec[sel_vec == 1])

            for (i in 1:length(res$selinf)) {
                if (res$selinf[[i]]["pval"] < 0.05) sel_with_selinf[only_sel[i]] <- 1
            }
            
            lasso_metrics <- metrics(sel_vec,c(1,beta!=0))
            selinf_metrics <- metrics(sel_with_selinf,c(1,beta!=0))

            num_sel_lasso <- sum(sel_vec)
            num_sel_selinf <- sum(sel_with_selinf)

            results_df[nrow(results_df) + 1,] <- c(SNR, prop_relevant, lasso_metrics$tpr, selinf_metrics$tpr, lasso_metrics$fdr, selinf_metrics$fdr, num_sel_lasso, num_sel_selinf)

            setTxtProgressBar(pb,nrow(results_df))

        }
    }
}

close(pb)

  |                                                                      |   0%

boundary (singular) fit: see help('isSingular')

Warning message:
"Model failed to converge with 1 negative eigenvalue: -9.5e+00"
boundary (singular) fit: see help('isSingular')



  |                                                                      |   1%

boundary (singular) fit: see help('isSingular')



  |=                                                                     |   2%

...

  |===================================================================== |  99%

boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



  |======================================================================|  99%

boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



  |======================================================================| 100%


In [8]:
write.csv(results_df, "C:/Users/matteda/OneDrive - Universitetet i Oslo/Skrivebord/phd/ConditioningApproach/new_minmod_simulation_results_SNR4.csv", row.names=FALSE)

In [31]:
if(q>0) rf = paste("+", paste("X", 1:q, sep="", collapse='+'),sep="") else rf = ''

In [34]:
setwd("c:/Users/matteda/OneDrive - Universitetet i Oslo/Skrivebord/phd/ConditioningApproach")

In [37]:
library(RhpcBLASctl)

In [38]:
RhpcBLASctl::blas_set_num_threads(20)

In [7]:
datas <- lapply(1:num_iter, function(j){data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=corr)})

In [ ]:
source('data_generator.r')
source('helper_functions.r')
source('main_functions.r')
source('sampling_functions.r')
source('selfmade.r')
source('selinf_functions.r')
source('testvector_functions.r')
source('utils.r')


RhpcBLASctl::blas_set_num_threads(20)

NewminMod = TRUE
if(NewminMod) source('minMod_modif.r')


n_subjects= 25
n_observations = 4
n = n_subjects*n_observations
p = 100
q = 1

varForSampling = 'minMod'

SNR_values = c(4)
prop_relevant_values = c(0.1, 0.15, 0.2)
corrs = c(0.2, 0.4, 0.6)
num_iter = 50

fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
)

if(q>0) rf = as.formula(paste(" ~ 1 +",paste("X", 1:q, sep="", collapse='+'),sep="")) else rf = as.formula(" ~ 1")

rand_form = list(subjects = rf)

tot_its <- length(SNR_values)*length(prop_relevant_values)*length(corrs)*num_iter

columns = c('SNR','prop_relevant', 'corr', 'TPR', 'FDR', 'num_sel')
results_df = data.frame(matrix(nrow = 0, ncol = length(columns))) 
colnames(results_df) = columns

for(SNR in SNR_values){
    for(prop_relevant in prop_relevant_values){
        for(corr in corrs){

            datas <- lapply(1:num_iter, data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=corr))

                data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=corr)
                X <- data$X
                subjects <- data$subjects
                y <- data$y
                beta <- data$beta

                # fixing lambda
                lambda = max(abs(t(X) %*% y)) / 2
                

                selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

                # Now we can define the function checking the congruency
                # with the original selection
                checkFun <- function(yb){

                    all(selFun(yb)$vec == sel_vec)

                }

                
                sel <- selFun(y)
                sel_vec <- sel$vec
                sel_names <- sel$names

                if(q>0) rf = paste("+", paste("X", 1:q, sep="", collapse='+'),sep="") else rf = ""

                sel_form = as.formula(
                paste("y ~ ", paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1 ", rf, "|subjects)")
                )

                final_model = lmer(formula = sel_form, data = data.frame(X, subjects, y))
                
                # and compute valid p-values conditional on the selection
                # (this takes some time and will produce a lot of warnings)
                suppressWarnings(res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = varForSampling,
                            checkFun = checkFun, nrSamples = 100, trace = FALSE))

                sel_with_selinf <- c(rep(0,p+1))
                names(sel_with_selinf) <- names(sel_vec)
                only_sel <- names(sel_vec[sel_vec == 1])

                for (i in 1:length(res$selinf)) {
                    if (res$selinf[[i]]["pval"] < 0.05) sel_with_selinf[only_sel[i]] <- 1
                }

                selinf_metrics <- metrics(sel_with_selinf,c(1,beta!=0))

                num_sel_selinf <- sum(sel_with_selinf)

                results_df[nrow(results_df) + 1,] <- c(SNR, prop_relevant, corr, selinf_metrics$tpr, selinf_metrics$fdr, num_sel_selinf)

            })
        }
    }
}


results_name = paste('simres_n',n,'_p',p,'_q',q,'_',varForSampling, sep='')
if(NewminMod) results_name = paste(results_name, '_new', sep='')
results_name = paste(results_name,'.csv', sep='')


write.csv(results_df, results_name, row.names=FALSE)

In [9]:
n_subjects= 25
n_observations = 4
p = 100
q = 1

SNR_values = c(4)
prop_relevant_values = c(0.1, 0.2, 0.3)
num_iter = 50

fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
)

rand_form = list(subjects =~ 1 + X1)

tot_its <- length(SNR_values)*length(prop_relevant_values)*num_iter

columns = c('SNR','prop_relevant', 'TPR_lasso', 'TPR_selinf', 'FDR_lasso', 'FDR_selinf', 'num_sel_lasso', 'num_sel_selinf')
results_df = data.frame(matrix(nrow = 0, ncol = length(columns))) 
colnames(results_df) = columns

pb = txtProgressBar(min = 0, max = tot_its, initial = 0, style = 3)

for(SNR in SNR_values){
    for(prop_relevant in prop_relevant_values){
        for(i in 1:num_iter){

            data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant)
            X <- data$X
            subjects <- data$subjects
            y <- data$y
            beta <- data$beta

            # fixing lambda
            lambda = max(abs(t(X) %*% y)) / 4
            

            selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

            # Now we can define the function checking the congruency
            # with the original selection
            checkFun <- function(yb){

                all(selFun(yb)$vec == sel_vec)

            }

            
            sel <- selFun(y)
            sel_vec <- sel$vec
            sel_names <- sel$names

            sel_form = as.formula(
            paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1 + X1|subjects)")
            )

            final_model = lmer(formula = sel_form, data = data.frame(X, subjects, y))
            
            # and compute valid p-values conditional on the selection
            # (this takes some time and will produce a lot of warnings)
            suppressWarnings(res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = 'minMod',
                        checkFun = checkFun, nrSamples = 100, trace = FALSE))

            sel_with_selinf <- c(rep(0,p+1))
            names(sel_with_selinf) <- names(sel_vec)
            only_sel <- names(sel_vec[sel_vec == 1])

            for (i in 1:length(res$selinf)) {
                if (res$selinf[[i]]["pval"] < 0.05) sel_with_selinf[only_sel[i]] <- 1
            }
            
            lasso_metrics <- metrics(sel_vec,c(1,beta!=0))
            selinf_metrics <- metrics(sel_with_selinf,c(1,beta!=0))

            num_sel_lasso <- sum(sel_vec)
            num_sel_selinf <- sum(sel_with_selinf)

            results_df[nrow(results_df) + 1,] <- c(SNR, prop_relevant, lasso_metrics$tpr, selinf_metrics$tpr, lasso_metrics$fdr, selinf_metrics$fdr, num_sel_lasso, num_sel_selinf)

            setTxtProgressBar(pb,nrow(results_df))

        }
    }
}

close(pb)

  |=                                                                     |   1%

boundary (singular) fit: see help('isSingular')



  |===                                                                   |   4%

boundary (singular) fit: see help('isSingular')



  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

...

  |===================================================================== |  99%

boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



  |======================================================================|  99%

boundary (singular) fit: see help('isSingular')



  |======================================================================| 100%


In [10]:
write.csv(results_df, "C:/Users/matteda/OneDrive - Universitetet i Oslo/Skrivebord/phd/ConditioningApproach/new_minmod_simulation_results_SNR4_smaller_lambda.csv", row.names=FALSE)